In [2]:
import re
import requests
import bs4
from random import randint
from time import sleep
from tqdm import tqdm
import pandas as pd

In [36]:
titles_st = []
with open('./dowloaded.txt') as file:
    line = file.readline()
    while line:
        modified_title = ''
        new_line = re.sub(r'[^\w\s]', '', line) 
        segmented = new_line[:-1].lower().split(' ')
        
        if segmented[-1] == 'the':
            modified_title = f'{segmented[-1]}-'
            
            for i in range(len(segmented)):
                if i == len(segmented)-1:
                    break
                
                modified_title = f'{modified_title}{segmented[i]}-'
        else:
            for i in range(len(segmented)):
                modified_title = f'{modified_title}{segmented[i]}-'

        titles_st.append(modified_title[:-1])
        line = file.readline()

In [145]:
info = pd.DataFrame(columns=['movie','site'])

for i in tqdm(range(1,1780)):
    url = f'https://subslikescript.com/movies?page={i}'
    response = requests.get(url)
    soup = bs4.BeautifulSoup(response.content,'lxml')
    
    websites = []
    names = []
    for movie in soup.find("ul", {"class": "scripts-list"}).find_all('a'):
        websites.append(movie['href'])
        names.append(movie.text)
    
    temp = pd.DataFrame({'movie' : names,'site' : websites})
    info = pd.concat([info,temp], ignore_index = True)
    
    if i%10 == 0:
        info.to_csv(f'./checkpoints/{i:04}.csv',index=False)
    
    sleep(randint(1,3))    

100%|████████████████████████████████████████████████████████████████████████████| 1779/1779 [1:18:31<00:00,  2.65s/it]


In [24]:
info.to_csv('./all_movies.csv',index=False)

In [26]:
info = pd.read_csv('./all_movies.csv')

In [27]:
info

,movie,site
0,Blind Detective (2013),movie/Blind_Detective-2332707
1,Killer Workout (1987),movie/Killer_Workout-91339
2,Budapest Noir (2017),movie/Budapest_Noir-5161018
3,Firefly (2013),movie/Firefly-3582840
4,Warriors of the Rainbow: Seediq Bale II (2011),movie/Warriors_of_the_Rainbow_Seediq_Bale_II-4...
...,...,...
53365,78/52: Hitchcock's Shower Scene (2017),movie/7852_Hitchcocks_Shower_Scene-4372240
53366,The Predator (2018),movie/The_Predator-3829266
53367,Blindspotting (2018),movie/Blindspotting-7242142
53368,Robin Hood The Rebellion (2018),movie/Robin_Hood_The_Rebellion-7052244


In [5]:
def write_sub_to_file(subtitle, file_name):
    
    file = open(file_name,'w',encoding="utf-8")
    for br in subtitle.find_all("br"):
        br.replace_with("\n")
        
    file.write(re.sub(r'\n+', '\n', subtitle.text))
    file.close()
    
def write_summary_to_file(summary, file_name):
    file = open(file_name,'w',encoding="utf-8")
    file.write(summary.text)
    file.close()

In [22]:
no_summaries = []
no_subs = []

In [23]:
base = 'https://subslikescript.com/'
sub_base = './subtitles/'
sum_base = './summary/'



for i in tqdm(range(len(info))):
    temp = re.sub(r'[^\w\s]', '', info['movie'][i])
    current_file = temp.replace(' ','_')
    
    sub_file = sub_base + current_file + '.txt'
    sum_file = sum_base + current_file + '.txt'
    
    url = base + info['site'][i]
    
    try:
        response = requests.get(url,timeout = 1)
    except  requests.exceptions.RequestException:
        no_subs.append(info['movie'][i])
        continue

    soup = bs4.BeautifulSoup(response.content,'lxml')
    
    subtitle = soup.find('div',{"class": "full-script"})
    summary = soup.find('p',{'class':'plot'})
    
    
    
    if subtitle:
        write_sub_to_file(subtitle,sub_file)
    else:
        no_subs.append(info['movie'][i])
        continue
    
    if summary:
        write_summary_to_file(summary, sum_file)
    else:
        no_summaries.append(info['movie'][i])

100%|█████████████████████████████████████████████████████████████████████████| 51858/51858 [11:09:18<00:00,  1.29it/s]


In [117]:
with open('./no_summaries.txt','w',encoding="utf-8") as file:
    for summ in no_summaries:
        file.write(summ+'\n')
    file.close()

In [115]:
with open('./no_subs.txt','w',encoding="utf-8") as file:
    for sub in no_subs:
        file.write(sub+'\n')
    file.close()

In [128]:
import os 

In [172]:
def compare(list1, list2):
    result = list()
    for i in tqdm(list1):
        temp = re.sub(r'[^\w\s]', '', i)
        current_file = temp.replace(' ','_')

        if current_file+'.txt' not in list2:
            result.append(i)
    return result

In [173]:
not_downloaded = compare(list(info['movie']),os.listdir('./subtitles/'))

100%|██████████████████████████████████████████████████████████████████████████| 53370/53370 [00:15<00:00, 3353.08it/s]


In [179]:
not_downloaded

['Puss in Boots: The Last Wish (2022)',
 'Big Trip 2: Special Delivery (2022)',
 'Iliza Shlesinger: Hot Forever (2022)',
 'Who Killed Cooper Dunn? (2022)',
 'Honk for Jesus. Save Your Soul. (2022)',
 'Nope (2022)',
 'Day Shift (2022)',
 'Not Okay (2022)',
 'Insecure (2014)',
 'The Giants (2011)',
 'Boris Without Béatrice (2016)',
 'Glue (2006)',
 'Garçon stupide (2004)',
 'Baduk (1992)',
 'The Little Chimney Sweep (1935)',
 'Der scheintote Chinese (1928)',
 'No Regret (2006)',
 'Atlas Shrugged: Who Is John Galt? (2014)',
 'Gallipoli (1981)',
 'Summer Wars (2009)',
 'Time Lapse (2014)',
 'Closed Circuit (2013)',
 'Such Good People (2014)',
 'Moon 44 (1990)',
 'Dark Places (2015)',
 'The Last Snows of Spring (1973)',
 'Furious 7 (2015)',
 'Admiral (2015)',
 'Beyond the Lights (2014)',
 'The Abduction of Zack Butterfield (2011)',
 'Frozen Fever (2015)',
 'Hot Tub Time Machine (2010)',
 'Heropanti (2014)',
 'Descendants (2015)',
 'Far from the Madding Crowd (2015)',
 'The Rules of the Game

In [178]:
base = 'https://subslikescript.com/'
sub_base = './_12subtitles/'
sum_base = './_12summary/'

temp_no_subs = []
temp_no_summs = []

for movie_name in tqdm(not_downloaded):

    temp = re.sub(r'[^\w\s]', '', movie_name)
    current_file = temp.replace(' ','_')
    
    sub_file = sub_base + current_file + '.txt'
    sum_file = sum_base + current_file + '.txt'
    
    site_loc = info.loc[info['movie'] == movie_name]
    
    url = base + site_loc.values[0][1]
    try:
        response = requests.get(url,timeout = 1)
    except  requests.exceptions.RequestException:
        temp_no_subs.append(movie_name)
        continue
        
    soup = bs4.BeautifulSoup(response.content,'lxml')
    
    subtitle = soup.find('div',{"class": "full-script"})
    summary = soup.find('p',{'class':'plot'})
    
    
    
    if subtitle:
        write_sub_to_file(subtitle,sub_file)
    else:
        temp_no_subs.append(movie_name)
        continue
    
    if summary:
        write_summary_to_file(summary, sum_file)
    else:
        temp_no_summs.append(movie_name)

  1%|█▏                                                                                 | 1/70 [00:02<02:38,  2.30s/it]


KeyboardInterrupt: 